In [2]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

def load_train(path):    
    datagen = ImageDataGenerator(rescale = 1/255.) 
    
    train_datagen_flow = datagen.flow_from_directory(
        path,
        target_size=(150, 150),
        batch_size=16,
        class_mode='sparse',
        subset='training',
        seed=12345)

    return train_datagen_flow

def create_model(input_shape=(150, 150, 3)):
    backbone = ResNet50(input_shape=input_shape,
                        weights='imagenet', 
                        include_top=False)
    backbone.trainable = True
    
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(12, activation='softmax'))
    
    optimizer = Adam(lr=0.001)
    
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['acc']
    )
    
    return model
    
def train_model(
    model,
    train_data,
    test_data,
    batch_size=None,
    epochs=5,
    steps_per_epoch=None,
    validation_steps=None,
    ):
    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)
        
    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size,
              epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)

    return model

# Output

Final accuracy: **`0.9955`**

```python
Train for 1463 steps, validate for 488 steps

Epoch 1/5

2023-05-31 03:22:40.425539: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10

2023-05-31 03:22:40.684281: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7

1463/1463 - 162s - loss: 0.5180 - acc: 0.8330 - val_loss: 0.3979 - val_acc: 0.8854

Epoch 2/5

1463/1463 - 154s - loss: 0.1076 - acc: 0.9685 - val_loss: 0.1983 - val_acc: 0.9381

Epoch 3/5

1463/1463 - 154s - loss: 0.0661 - acc: 0.9797 - val_loss: 0.0562 - val_acc: 0.9833

Epoch 4/5

1463/1463 - 154s - loss: 0.0606 - acc: 0.9821 - val_loss: 0.0260 - val_acc: 0.9927

Epoch 5/5

1463/1463 - 154s - loss: 0.0465 - acc: 0.9876 - val_loss: 0.0173 - val_acc: 0.9955

WARNING:tensorflow:sample_weight modes were coerced from

  ...

    to  

  ['...']

488/488 - 38s - loss: 0.0173 - acc: 0.9955

```